In [1]:
import os
from datetime import datetime

drive = True

if drive:
    from google.colab import drive
    drive.mount('/content/drive')

    %cd /content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob

    path = "/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob/"

    # check python file folder
    assert os.path.exists(path + "python"), f"Upload python files in {path}python"
    %cd ./python

    # check data folder
    assert os.path.exists(path + "data"), f"Upload data files in {path}data"
else:
    path = "../"

    # check python file folder
    assert os.path.exists(path + "python"), f"Upload python files in {path}python"

    # check data folder
    assert os.path.exists(path + "data"), f"Upload data files in {path}data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob
/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob/python


In [2]:
import myk_data
import myk_models
import myk_loss
import myk_train
import myk_evaluate

import torch
from torch.utils.data import DataLoader

In [3]:
from torch.utils.tensorboard import SummaryWriter

In [4]:
# used for the writing of example outputs
run_name="audio_lstm_lpf"
# dataset : need an input and output folder in this folder
audio_folder = f"../data/{run_name}"
#audio_folder = "../../data/audio_ht1"
assert os.path.exists(audio_folder), f"Audio folder  not found. Looked for {audio_folder}"
# used to render example output during training
test_file = "../data/guitar.wav"
assert os.path.exists(test_file), "Test file not found. Looked for " + test_file

In [5]:
# initialize net specs
lstm_hidden_size = 64
learning_rate = 5e-3
batch_size = 50
max_epochs = 10000

# create the logger for tensorboard
writer = SummaryWriter()

In [6]:
print("Loading dataset from folder ", audio_folder)
dataset = myk_data.generate_dataset(audio_folder + "/input/", audio_folder + "/output/", frag_len_seconds=0.5)

print("Splitting dataset")
train_ds, val_ds, test_ds = myk_data.get_train_valid_test_datasets(dataset)

Loading dataset from folder  ../data/audio_lstm_lpf
loading input and output of ht1
    loading output of ht1
    generate_dataset:: Loaded frames from audio file 22050
    found input fragments of shape (120, 22050, 1)
    found output fragments of shape (120, 22050, 1)
    total input shape: (120, 22050, 1)
    total output shape: (120, 22050, 1)

total input tensor shape torch.Size([120, 22050, 1])
total output tensor shape torch.Size([120, 22050, 1])
Splitting dataset


In [7]:
dataset[0]

(tensor([[-3.0518e-05],
         [-3.0518e-05],
         [ 0.0000e+00],
         ...,
         [ 0.0000e+00],
         [ 0.0000e+00],
         [ 0.0000e+00]]),
 tensor([[ 0.0000e+00],
         [ 0.0000e+00],
         [ 0.0000e+00],
         ...,
         [-1.4067e-05],
         [-1.3947e-05],
         [-1.3947e-05]]))

In [8]:
# test GPU, must be done after splitting
device = myk_train.get_device()

cuda device available


In [9]:
# create data loaders
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))

In [12]:
model = myk_models.SimpleLSTM(hidden_size=lstm_hidden_size).to(device)
print(model)

SimpleLSTM(
  (lstm): LSTM(1, 64, batch_first=True)
  (dense): Linear(in_features=64, out_features=1, bias=True)
)


In [13]:
# crate optimizer and loss function
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min', factor=0.5, patience=5, verbose=True) non lo usava

loss_functions = myk_loss.LossWrapper()

# https://github.com/Alec-Wright/Automated-GuitarAmpModelling/blob/main/dist_model_recnet.py
# https://github.com/Alec-Wright/CoreAudioML/blob/bad9469f94a2fa63a50d70ff75f5eff2208ba03f/training.py

In [14]:
#%load_ext tensorboard
#%tensorboard --logdir logs

In [15]:
# training loop
lowest_val_loss = 0
best_loss = False

max_patience = 500
curr_patience = max_patience

# datetime object containing current date and time
best_epoch = 0
now = datetime.now()
dt_string = now.strftime(f"models/{model.model_type}_%d-%m-%Y_%H-%M-%S")

os.mkdir(os.path.join(".", dt_string))

for epoch in range(max_epochs):
    #ep_loss = myk_train.train_epoch_interval(model, train_dl, loss_functions, optimiser, device=device)
    ep_loss = myk_train.train_epoch_interval(model, train_dl, loss_functions, optimiser, device=device)

    #ep_loss = myk_train.train_epoch(model, train_dl, loss_functions, optimiser, device=device)
    val_loss = myk_train.compute_batch_loss(model, val_dl, loss_functions, device=device)
    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar("Loss/train", ep_loss, epoch)

    # check if we have beaten our best loss to date
    if lowest_val_loss == 0:# first run
        lowest_val_loss = val_loss
    elif val_loss < lowest_val_loss:# new record
        lowest_val_loss = val_loss
        best_loss = True
    else: # no improvement
        best_loss = False
        curr_patience -= 1

    if best_loss: # save best model so far
        best_epoch = epoch
        print(f"    Record loss - saving at epoch {epoch}")
        # save for RTNeural
        model.save_for_rtneural(f"{dt_string}/model.json")
        # save for pythorch
        torch.save(model.state_dict(), f"{dt_string}/model.ph")
        print(f"    epoch {epoch}, train_loss {ep_loss}, val_loss {val_loss} ")
        curr_patience = max_patience
    if epoch % 50 == 0: # save an example processed audio file
        myk_evaluate.run_file_through_model(model, test_file, audio_folder + "/" + run_name + str(epoch)+".wav")
        print(f"epoch {epoch}, train_loss {ep_loss}, val_loss {val_loss} ")
    if curr_patience == 0:
        print("max patience reached, stopping training")
        # load best parameters in the model
        model.load_state_dict(torch.load(f"{dt_string}/model.ph"))
        model.eval() # set inference state in the possible layers that need it
        myk_evaluate.run_file_through_model(model, test_file, audio_folder + "/" + run_name + str(best_epoch)+"_BEST.wav")
        break

epoch 0, train_loss 0.7276068925857544, val_loss 0.6470811367034912 
    Record loss - saving at epoch 1
    epoch 1, train_loss 0.6044882535934448, val_loss 0.5986725687980652 
    Record loss - saving at epoch 2
    epoch 2, train_loss 0.596645176410675, val_loss 0.5832301378250122 
    Record loss - saving at epoch 3
    epoch 3, train_loss 0.5917901396751404, val_loss 0.5825192332267761 
    Record loss - saving at epoch 4
    epoch 4, train_loss 0.5860413312911987, val_loss 0.581730306148529 
    Record loss - saving at epoch 6
    epoch 6, train_loss 0.5816167593002319, val_loss 0.5811533331871033 
    Record loss - saving at epoch 9
    epoch 9, train_loss 0.5756134390830994, val_loss 0.5789920091629028 
    Record loss - saving at epoch 10
    epoch 10, train_loss 0.5901691317558289, val_loss 0.5756321549415588 
    Record loss - saving at epoch 17
    epoch 17, train_loss 0.5668160915374756, val_loss 0.5739730596542358 
    Record loss - saving at epoch 20
    epoch 20, train_

In [16]:
to_append = [f'\n{run_name}/*.wav', f'\n!{run_name}/*_BEST.wav']

with open(f'{audio_folder}/../.gitignore', "r+") as gitignore:
    read = gitignore.read()
    if to_append[0][2:len(to_append[0])] not in read:
        gitignore.writelines(to_append)